# Beautiful Soup Lyric Retrieval
## GOAL:
#### Retrieve lyrics from the links populated in the 'lyricurl' column of song_data.csv

In [1]:
# Import Libraries
import pandas as pd
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
import os
import re

In [2]:
data = pd.read_csv('data/song_data.csv', index_col=0)

In [3]:
data.head()

,Artist,Song,SongID,Popularity,Lyrics,href,lyricurl
0,Miley Cyrus,Flowers,0yLdNVWF3Srea0uzk55zFn,87,NaN,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://genius.com/Miley-cyrus-flowers-lyrics
1,Metro Boomin,Creepin' (with The Weeknd & 21 Savage),2dHHgzDwk4BJdRwy9uXhTO,97,NaN,https://api.spotify.com/v1/tracks/2dHHgzDwk4BJ...,https://genius.com/Metro-boomin-the-weeknd-and...
2,SZA,Kill Bill,1Qrg8KqiBpW07V7PNxwwwL,93,NaN,https://api.spotify.com/v1/tracks/1Qrg8KqiBpW0...,https://genius.com/Sza-kill-bill-lyrics
3,Central Cee,LET GO,3zkyus0njMCL6phZmNNEeN,94,NaN,https://api.spotify.com/v1/tracks/3zkyus0njMCL...,https://genius.com/Central-cee-let-go-lyrics
4,Tiësto,10:35,6BePGk3eCan4FqaW2X8Qy3,90,NaN,https://api.spotify.com/v1/tracks/6BePGk3eCan4...,https://genius.com/Tiesto-and-tate-mcrae-10-35...


In [4]:
data['lyricurl'][0]

'https://genius.com/Miley-cyrus-flowers-lyrics'

In [29]:
for i in range(len(data)):
    url = data.at[i, 'lyricurl']
    
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'}) # This will disguise the bs4 object as a browser to avoid a 403 error
    
    webpage = urlopen(req).read()
    
    page_soup = soup(webpage, 'html.parser')
    
    containers = page_soup.find_all(re.compile("span$Referent"))
    for container in containers:
        print(container)

# **Pending stackoverflow help**